In [4]:
import pandas as pd
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# TÁI LẬP TRÌNH TỪ BÀI BÁO GỐC

Sử dụng siêu tham số từ bài báo gốc
Sử dụng chương trình trích xuất đặc trưng được cung cấp từ bài báo gốc

In [ ]:
import os
os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\_original\Human")

label = pd.read_csv("label.csv")
# print(label)
feat_a = pd.read_csv("total_features_a.csv")
# print(feat_a)
feat_b = pd.read_csv("total_features_b.csv")
# print(feat_b)
feat_all = pd.concat([feat_a, feat_b], axis=1)
print(feat_all.shape)

X_no_enc = feat_all.values
y_no_enc = label.values

In [ ]:
from tensorflow.keras import Model
def AE_net(n_col, k = 208):
    model_AE = Sequential(
        [
            Dense(k, activation = 'sigmoid', input_shape=(n_col,)),
            Dropout(rate = 0.2),
            Dense(n_col, activation = 'sigmoid', input_shape=(k,))
        ]
    )

    # Constructing Autoencoder
    model_AE.compile(optimizer='nadam',
                      loss= 'mean_squared_error',
                      metrics = "accuracy")

    # model_AE1.summary()
    enc_net = Model(inputs=model_AE.input,
                    outputs=model_AE.layers[1].output)
    # enc_net.summary()
    return model_AE, enc_net

In [ ]:
#Initialize the Autoencoder for chain A. B
k = 208
n_col = feat_a.shape[1]
model_AE1, enc_a = AE_net(n_col, k)

# Fitting the Autoencoder
print("\nHuan luyen AE 1...")
print(f"feat_a {feat_a.shape}")
history_a = model_AE1.fit(feat_a, feat_a,
                          epochs=200,
                          batch_size=50,
                          validation_split = 0.2,
                          verbose=0)

feat_a_enc = enc_a.predict(feat_a)
print("feat_a_enc", feat_a_enc.shape)


In [ ]:
print("\nHuan luyen AE 2...")
print(f"feat_b {feat_b.shape}")
model_AE2, enc_b = AE_net(n_col, k)
history_b = model_AE2.fit(feat_b, feat_b,
                          epochs=200,
                          batch_size=50,
                          validation_split = 0.2,
                          verbose=0)

feat_b_enc = enc_b.predict(feat_b)
print(feat_b_enc.shape)

# --- Save
from utils import pickleplus
pickleplus.dump(feat_b_enc, "human_enc_feat_b.pkl")

# SỬ DỤNG LẠI MÔ HÌNH ĐÃ ĐƯỢC HUẤN LUYỆN CỦA BÀI BÀO GỐC

In [5]:
import os

os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\_original\Human")

label = pd.read_csv("label.csv")
feat_a = pd.read_csv("total_features_a.csv")
feat_b = pd.read_csv("total_features_b.csv")
feat_all = pd.concat([feat_a, feat_b], axis=1)
print(feat_all.shape)

(8161, 1232)


In [6]:
from tensorflow.keras.models import load_model
enc_a = load_model("AE_Human_a_.h5")
print(enc_a.summary())

X_no_enc = feat_all.values
y_no_enc = label.values

feat_a_enc = enc_a.predict(feat_a)
print(feat_a_enc.shape)

enc_b = load_model("AE_Human_b_.h5")
print(enc_b.summary())

feat_b_enc = enc_b.predict(feat_b)
print(feat_b_enc.shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 208)               128336    
_________________________________________________________________
dropout (Dropout)            (None, 208)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 616)               128744    
Total params: 257,080
Trainable params: 257,080
Non-trainable params: 0
_________________________________________________________________
None
(8161, 616)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 208)               128336    
_________________________________________________________________
dropout_1 (Dropout)          (None, 208)               0         
___________

In [7]:
import numpy as np

feat_all_enc = np.concatenate((feat_a_enc, feat_b_enc), axis=1)
print('Human new trainning set', feat_all_enc.shape)

Human new trainning set (8161, 1232)


In [8]:
import numpy as np

X_tr = np.copy(feat_all_enc)
y_tr = np.copy(label)
y_tr[y_tr == -1] = 0
print(y_tr)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [13]:
import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline

gbm = lgb.LGBMClassifier(
    n_estimators=500, # 500
    num_leaves=80,  # 80
    learning_rate=0.05
    # max_depth=10,
    # feature_fraction=0.9,
    # bagging_fraction=0.8,
    # bagging_freq=5,
)

# gbm = lgb.LGBMClassifier(**params)

# pipeline = Pipeline([('transformer', t), ('estimator', gbm)])
cv = StratifiedKFold(n_splits=5, random_state=48, shuffle=True)
print(f'\n{cv.__str__()}...')
scr = cross_val_score(estimator=gbm, scoring=['auc'], X=X_tr, y=y_tr, cv=cv, n_jobs=2)


StratifiedKFold(n_splits=5, random_state=48, shuffle=True)...


ValueError: For evaluating multiple scores, use sklearn.model_selection.cross_validate instead. ['auc'] was passed.

In [10]:
print(scr)
print("mean {:0.4f}, std {:0.3f}".format(np.mean(scr), np.std(scr)))

[0.9816289  0.98651961 0.98223039 0.9810049  0.98835784]
mean 0.9839, std 0.003


In [11]:
import lightgbm as lgb

gbm = lgb.LGBMClassifier(
    n_estimators=500,
    num_leaves=80,
    learning_rate=0.05
)

gbm.fit(X_tr, y_tr.ravel())

LGBMClassifier(learning_rate=0.05, n_estimators=500, num_leaves=80)

# TEST TRÊN TẬP ĐỘC LẬP

In [ ]:
import os
os.chdir(r"D:\NCSI\3 - Thuc nghiem\ModifiedModel\AE_LGBM_2020\mod_AE_LGBM\Independent Species\Ecoli")
feat_a = pd.read_csv("total_features_a.csv")
feat_a = enc_a.predict(feat_a)

feat_b = pd.read_csv("total_features_b.csv")
feat_b = enc_b.predict(feat_b)

data_te = np.concatenate((feat_a, feat_b), axis=1)
print('data_te', data_te.shape)
label_te = [1] * data_te.shape[0]

In [ ]:
print(gbm.score(data_te, label_te))
print("{:.4f}".format(gbm.score(data_te, label_te)))